In [54]:
import pennylane as qml
num_wires = 5
if 1:
    from qiskit_ibm_provider import IBMProvider
    provider = IBMProvider()
    backend = provider.get_backend('ibm_hanoi')
    dev = qml.device('qiskit.ibmq', wires=num_wires, backend=backend, shots=200)
else:
    dev = qml.device('default.qubit', wires=num_wires)

def ghz(dev):
    @qml.qnode(dev)
    def circuit():
        n=num_wires
        qml.Hadamard(wires=0)
        for i in range(1, n):  
            qml.CNOT(wires=[0,i])
        return qml.probs(wires=range(n))
    return circuit
circ = ghz(dev)
print('backend:',backend)
print(qml.draw(circ)()) 

backend: <IBMBackend('ibm_hanoi')>
0: ──H─╭●─╭●─╭●─╭●─┤ ╭Probs
1: ────╰X─│──│──│──┤ ├Probs
2: ───────╰X─│──│──┤ ├Probs
3: ──────────╰X─│──┤ ├Probs
4: ─────────────╰X─┤ ╰Probs


In [55]:
probTens = circ()  # run circuit on IBM device
print('probs:',probTens)
jid=dev._current_job.job_id()
print('IBM job ID',jid) 

probs: [0.455 0.    0.005 0.    0.005 0.    0.005 0.005 0.015 0.    0.    0.
 0.005 0.005 0.    0.01  0.01  0.    0.    0.01  0.    0.    0.    0.025
 0.    0.005 0.    0.01  0.    0.    0.015 0.415]
IBM job ID cpxcrvvn8rag008qk2y0


In [56]:
job = provider.retrieve_job(jid)
# Retrieve the results from the job
jobRes = job.result()
resL=jobRes.results  
nqc=len(resL)  # number of circuit in the job
counts=jobRes.get_counts()
if nqc==1: counts=[counts]  # this is poor design
print('M: got %d circ+results'%nqc, 'JID=',jid)

M: got 1 circ+results JID= cpxcrvvn8rag008qk2y0


In [59]:
for ic in range(nqc):
    qc=job.circuits()[ic]  # transpiled circuit 
    resHead=resL[ic].header # auxiliary info about used hardware
    print('\nM: circ=%d %s'%(ic,qc.name))
    print('counts:',counts[ic])
    q2_depth=qc.depth(filter_function=lambda x: x.operation.num_qubits > 1)
    print('circuit q2_depth=%d , gate count:'%q2_depth,dict(qc.count_ops()))
    print(qc.draw(output="text", idle_wires=False,fold=120))
    print('result header: ',end=''); print(resHead)



M: circ=0 circ0
counts: {'00010': 3, '10011': 1, '00000': 91, '11110': 2, '11101': 5, '00001': 2, '00100': 1, '01000': 1, '01100': 1, '00110': 1, '11011': 2, '01111': 3, '11111': 83, '11100': 1, '10110': 1, '11001': 2}
circuit q2_depth=11 , gate count: {'cx': 10, 'measure': 5, 'rz': 2, 'sx': 1, 'barrier': 1}
global phase: π/4
                                                                                              ┌───┐                    »
       Qubit(QuantumRegister(5, 'q'), 2) -> 0 ────────────────────────────────────────────────┤ X ├────────────────────»
                                                                               ┌───┐     ┌───┐└─┬─┘          ┌───┐     »
       Qubit(QuantumRegister(5, 'q'), 3) -> 1 ─────────────────────────────────┤ X ├──■──┤ X ├──■────■────■──┤ X ├──■──»
                                                                               └─┬─┘  │  └─┬─┘       │  ┌─┴─┐└─┬─┘┌─┴─┐»
Qubit(QuantumRegister(22, 'ancilla'), 3) -> 2 ────────────────────